# Problem klasyfikacji gęstości (DCP)

## Imports

In [1]:
# pip install pygad
# pip install joblib

In [52]:
import math
import time
# import pygad
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

In [33]:
def eca_get_lut(rule_num: int) -> np.ndarray:
    return np.array([int(x) for x in bin(rule_num)[2:].zfill(8)], dtype=np.uint8)

def eca_evolve(lut: np.ndarray, x: np.ndarray) -> np.ndarray:
    return lut[7 - (np.roll(x, 1) * 4 + x * 2 + np.roll(x, -1))]

def eca_evolve_spacetime(lut: np.ndarray, initial_conf: np.ndarray, steps: int) -> np.ndarray:
    rows = [initial_conf]
    for _ in range(1, steps):
        rows.append(eca_evolve(lut, rows[-1]))
    return np.stack(rows)


In [23]:
np.random.randint(2, size=9)

array([0, 0, 0, 1, 0, 1, 0, 1, 0])

In [24]:
lut = eca_get_lut(232)
x = np.array([0,1,1,1,1,1,0,0,0])
eca_evolve(lut, x)

array([0, 1, 1, 1, 1, 1, 0, 0, 0], dtype=uint8)

In [34]:
eca_evolve_spacetime(lut, x, 18)

array([[0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 0, 0]])

(1D ECA Simulator: https://elife-asu.github.io/wss-modules/modules/1-1d-cellular-automata/)

## Wczytywanie konfiguracji pana Marcina
Pliki z konfiguracjami można pobrać z https://github.com/D3M80L/CA/tree/master/phd/data.

In [36]:
def convert_configuration(configuration: bytes, N: int):
    binary_str = ''.join(f'{byte:08b}' for byte in configuration)
    return np.array([int(bit) for bit in binary_str[:N]])


In [37]:
def configuration_reader(file_name: str, N: int, negate: bool):
    bytes_per_configuration = math.ceil(N / 8)
    configurations = []

    with open(file_name, 'rb') as file:
        while True:
            bytes_read = file.read(bytes_per_configuration)
            if not bytes_read:
                break

            cfg = convert_configuration(bytes_read, N)
            configurations.append(cfg)
            
            if negate:
                configurations.append(1 - cfg)

    return np.stack(configurations)


---
## zad.9 (lekko zmodyfikowane)

Zapoznaj się z wynikami z artykułu:\
*Mendonça, J. R. G. (2019).* **Simply modified GKL density classifiers that reach consensus faster.** *Physics Letters A, 383(19), 2264–2266. 
doi:10.1016/j.physleta.2019.04.033* (https://sci-hub.se/https://www.sciencedirect.com/science/article/abs/pii/S0375960119303512?via=ihub)

Spróbuj powtórzyć eksperymenty tam wykonane dla wybranego N dla którego mamy "zbiór Marcina" % poprawnych klasyfikacji dla reguł GKL(j, k) dla kilku, wybranych przez Ciebie przypadków j, k.

**Wariant (*) - zbadaj również średni czas klasyfikacji - tzn. czy widać, które reguły są szybsze od innych.**

### Implementacja GKL(j, k)
$
x^{t+1}_i = 
\begin{cases} 
\text{maj}(x^t_{i-k}, x^t_{i-j}, x^t_i) & \text{jeśli } x^t_i = 0, \\
\text{maj}(x^t_i, x^t_{i+j}, x^t_{i+k}) & \text{jeśli } x^t_i = 1, 
\end{cases}
$

gdzie wybieramy sąsiadów $i\pm j$ oraz $i\pm k$ takich, że $k>j\geq i$.

In [15]:
def majority(p, q, r):
    return np.floor(0.5 * (p + q + r))

def gkl_step(x: np.ndarray, j: int, k: int) -> np.ndarray:
    x_left = np.roll(x, j)
    x_right = np.roll(x, -j)
    x_far_left = np.roll(x, k)
    x_far_right = np.roll(x, -k)

    new_x = np.where(x == 0, majority(x_far_left, x_left, x), majority(x, x_right, x_far_right))
    return new_x


In [19]:
def gkl_spacetime(initial_conf: np.ndarray, j: int, k: int, steps: int) -> np.ndarray:
    rows = [initial_conf]
    for _ in range(1, steps):
        rows.append(gkl_step(rows[-1], j, k))
    return np.stack(rows)


Dla k = -1, j = 1 otrzymujemy te same wyniki co w `eca_evolve()` i `eca_evolve_spacetime()` dla ECA 232 (majority):

In [32]:
gkl_step(x, -1, 1)

array([0., 1., 1., 1., 1., 1., 0., 0., 0.])

In [31]:
gkl_spacetime(x, -1, 1, 18)

array([[0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 0., 0., 0.]])

### Eksperyment dla N = 23

In [39]:
configurations = configuration_reader("ALL_N23.bin", 23, True)
len(configurations)

364724

Funkcja do klasyfikacji gęstości:

In [40]:
def dcp_gkl(initial_config, j, k, steps):
    initial_density = sum(initial_config)/len(initial_config)
    config = gkl_spacetime(initial_config, j, k, steps)[-1]
    result_density = sum(config)/len(config)

    if initial_density < 0.5:
        if result_density < 0.5:
            return 1
        else:
            return 0
    if initial_density > 0.5:
        if result_density > 0.5:
            return 1
        else:
            return 0
    return 0


In [49]:
neighbors = [(4,12), (3,9), (2,6), (5,15), (1,3), (1,9), (1,11), (2,14), (2,10), (3,15), (1,7), (1,5)]
results = []
times = []

for j, k in neighbors:
    print(f'GKL({j},{k})')

    start_time = time.time()
    l = Parallel(n_jobs=-1)(delayed(dcp_gkl)(config, j, k, 46) for config in configurations)
    end_time = time.time()
    t = end_time - start_time
    times.append(round(t, 5))
    
    results.append(round(sum(l)/len(configurations),5))
    print(f'Poprawna klasyfikacja gęstości: {100*results[-1]}%')
    print(f'Czas wykonania: {times[-1]} sekund')
    print()

GKL(4,12)
Poprawna klasyfikacja gęstości: 90.515%
Czas wykonania: 278.20585 sekund

GKL(3,9)
Poprawna klasyfikacja gęstości: 90.515%
Czas wykonania: 288.81525 sekund

GKL(2,6)
Poprawna klasyfikacja gęstości: 90.515%
Czas wykonania: 290.2187 sekund

GKL(5,15)
Poprawna klasyfikacja gęstości: 90.515%
Czas wykonania: 330.15819 sekund

GKL(1,3)
Poprawna klasyfikacja gęstości: 90.515%
Czas wykonania: 316.66102 sekund

GKL(1,9)
Poprawna klasyfikacja gęstości: 91.244%
Czas wykonania: 352.0376 sekund

GKL(1,11)
Poprawna klasyfikacja gęstości: 89.876%
Czas wykonania: 333.32634 sekund

GKL(2,14)
Poprawna klasyfikacja gęstości: 91.621%
Czas wykonania: 301.38556 sekund

GKL(2,10)
Poprawna klasyfikacja gęstości: 90.728%
Czas wykonania: 332.68055 sekund

GKL(3,15)
Poprawna klasyfikacja gęstości: 90.728%
Czas wykonania: 340.35098 sekund

GKL(1,7)
Poprawna klasyfikacja gęstości: 91.621%
Czas wykonania: 328.28764 sekund

GKL(1,5)
Poprawna klasyfikacja gęstości: 90.728%
Czas wykonania: 286.03217 sekund



In [51]:
df = pd.DataFrame({'correctness': results, 'time': times}, index=neighbors)
df.T

,"(4, 12)","(3, 9)","(2, 6)","(5, 15)","(1, 3)","(1, 9)","(1, 11)","(2, 14)","(2, 10)","(3, 15)","(1, 7)","(1, 5)"
correctness,0.90515,0.90515,0.90515,0.90515,0.90515,0.91244,0.89876,0.91621,0.90728,0.90728,0.91621,0.90728
time,278.20585,288.81525,290.21870,330.15819,316.66102,352.03760,333.32634,301.38556,332.68055,340.35098,328.28764,286.03217


---
# Algorytmy ewolucyjne

## zad.11

Zaimplementuj algorytm ewolucyjny, który poszukuje rozwiązań DCP wśród binarnych automatów komórkowych o promieniu sąsiedztwa r=2, dla liczby komórek N=23 (korzystając ze zbioru Marcina) i limitu czasu T = 2*N. Zdecyduj sama / sam ile populacji i jak duże populacje potrzebujesz - możesz to określić robiąc eksperymenty. Możesz również dowolnie manipulować mutacjami, krzyżowaniem, selekcją itd.

Jaki najlepszy wynik (największą liczbę poprawnie sklasyfikowanych konfiguracji) uda Ci się uzyskać? Podaj LUT najlepszej, znalezionej przez Twój algorytm reguły. 

Uzyskane przez nas wszystkich reguły porównam na zbiorach N=23 i większych, jak tylko otrzymam wszystkie rozwiązania i wyłonimy zwycięzców. Może uda się komuś z nas pobić GKL?

Sugestie:
- Do wstępnych testów możesz najpierw użyć mniejszego N i zamiast zbioru Marcina, użyć mniejszy zbiór wygenerowany losowo - program będzie działać szybciej, zatem szybciej będziesz wprowadzać w nim zmiany.
- Koniecznie użyj przetwarzania równoległego aby w pełni wykorzystać moc współczesnych procesorów oraz cache’owania - tak, żeby nie przeliczać tej samej reguły CA wiele razy.
- GitHub CodeSpaces dają 4-rdzeniowy procesor w chmurze dostępny za darmo (30 lub 60 godzin obliczeń w miesiącu).
- Czy dla każdego wyliczenia wartości funkcji dopasowania trzeba używać cały zbiór warunków początkowych? Może aby było szybciej, dla każdej populacji można losować pewien podzbiór zbioru warunków początkowych i dzięki temu szybciej tworzyć nowe populacje? (A może te podzbiory też powinny podlegać “ewolucji”? Mamy wtedy do czynienia z koewolucją.)
- Przedyskutuj to zagadnienie z ChatGPT. Poproś o sugestie, pomysły na usprawnienia kodu itd!

In [ ]:
# ECA to automaty o promieniu r=1, teraz mamy r=2, czyli sąsiedztwo 5!!
# trzeba zmodyfikować eca_get_lut(do 32 a nie 8) i eca_evolve(na filmiku)
# podać lut, który wyszedł jako najlepszy!!